# Install required libraries


In [1]:
!pip install scanpy
!pip install optuna
!pip install igraph

### Load required packages

In [2]:
import scanpy as sc
import numpy as np
import pandas as pd
import os
import sklearn
import optuna
sc.settings.verbosity = 1
print(f'sklearn_version:{sklearn.__version__}')

sklearn_version:1.5.2


In [3]:
dataset_type = "cancer" # neuro,cancer
dataset_folder = os.path.join(os.getcwd(),"data")
dataset_path  = os.path.join(dataset_folder,f'{dataset_type}.h5ad')
dataset_subpath = os.path.join(dataset_folder,f'{dataset_type}_hvg.h5ad')
dataset_pandas_path = os.path.join(dataset_folder,f'{dataset_type}_hvg.csv')
dataset_preaggregated_path = os.path.join(dataset_folder,f'{dataset_type}_preaggregated_.csv')
dataset_aggregated_path = os.path.join(dataset_folder,f'{dataset_type}_aggregated.csv')
dataset_train_path = os.path.join(dataset_folder,f'train_{dataset_type}.csv')
dataset_test_path = os.path.join(dataset_folder,f'test_{dataset_type}.csv')
DOWNLOAD= True
RANDOM_SEED = 38
dataset_links = {"neuro": "https://datasets.cellxgene.cziscience.com/2808a16d-64c5-451b-91a5-c1a2d9f270d5.h5ad ", "cancer": "https://datasets.cellxgene.cziscience.com/76c942bd-45c3-47ec-b290-1e695ec9c177.h5ad"}
if not os.path.isdir(dataset_folder):
  print(f'creating dataset_folder={dataset_folder}')
  os.makedirs(dataset_folder,exist_ok=True)


if not os.path.isfile(dataset_path) and DOWNLOAD:
  dataset_link = dataset_links[dataset_type]
  print(f'downloading dataset from link={dataset_link}.....')
  print(f"...and saving to {dataset_path}....")
  command = f"wget {dataset_link}  -O {dataset_path}"
  print(f'running command:{command}')
  os.system(command)



downloading dataset from link=https://datasets.cellxgene.cziscience.com/76c942bd-45c3-47ec-b290-1e695ec9c177.h5ad.....
...and saving to /content/data/cancer.h5ad....
running command:wget https://datasets.cellxgene.cziscience.com/76c942bd-45c3-47ec-b290-1e695ec9c177.h5ad  -O /content/data/cancer.h5ad


### Dataset preprocessing

### Helper functions


In [4]:

def process_categorical_columns(data = None, columns=[],category='ohe'):
  """
  This function encodes columns in data to either one hot encoding or label
  encoding
  Args:
  data (pd.Dataframe): preaggregated data e.g gene_exp_df
  columns (list of str) : columns to encode
  category (str): type of encoding e.g ohe or le
  returns:
  pd.DataFrame
  """
  cols_before = set(data.columns)
  if category == 'ohe':
    data = pd.get_dummies(data, columns=columns)
  elif category == 'le':
    new_columns = [f'{column}_encoded' for column in columns]
    data[new_columns] = data[columns].apply(lambda x: pd.factorize(x)[0])
    return data,new_columns
  else:
    raise Exception(f'invalid category={category}')
  encoded_columns = list(set(data.columns).difference(cols_before))
  data[encoded_columns] = data[encoded_columns].astype(np.int64)
  return data,encoded_columns

def compute_drop_columns(data=None,columns = [],null_threshold = 0.33,n=1000):
  """
  This function drops columns that have null percentage exceeding threshod
  encoding
  Args:
  data (pd.Dataframe): preaggregated data e.g gene_exp_df
  columns (list of str) : columns to consider for drop
  null_threshold (float): nul percentage to consider when dropping column
  n (int): sample size of dataframe
  returns:
  drop_columns : list of str
  """
  drop_columns = []
  for col in columns:
    nan_count = data[col].isna().sum()
    nan_percentage = nan_count/n
    if nan_percentage > 0:
      print(f'col={col} null_perc={nan_percentage}')
    if nan_percentage > null_threshold:
      print(f'dropping col={col}')
      drop_columns.append(col)
  return drop_columns

def handle_missing_data(data=None,numerical_columns=[],categorical_columns=[]):
  """
  This function replaces Nan with mean for numerical columns
  and with mode for categorical columns
  Args:
  data (pd.Dataframe): preaggregated data e.g gene_exp_df
  numerical_columns (list of str) : numerical columns to fill with mean
  categorical_columns (list of str) : categorical columns to fill with mode
  returns:
  pd.DataFrame
  """
  for col in numerical_columns:
    m = data[col].mean()
    data[col].fillna(m,inplace=True)
  for col in categorical_columns:
    m = data[col].mode()
    data[col].fillna(m,inplace=True)
  return data

#### Subset dataset

In [5]:
LOAD_COMPLETE_DATASET = False # High RAM needed ~50gb
#need to run LOAD_COMPLETE_DATASET=True to run LOAD_PREAGGREGATED_DATASET=True and LOAD_AGGREGATED_DATASET=True
LOAD_PREAGGREGATED_DATASET = False  and (not LOAD_COMPLETE_DATASET)# LOAD_COMPLETE_DATASET needs to be False and False if high RAM available
LOAD_AGGREGATED_DATASET = False and (not LOAD_PREAGGREGATED_DATASET)#LOAD_PREAGGREGATED_DATASET needs to be False and True if NOT high RAM available
# can run LOAD_TRAIN_TEST_DATASET independently of the rest as long as we have data/ folder in same directory as notebook with appropriate files
LOAD_TRAIN_TEST_DATASET = True and (not LOAD_AGGREGATED_DATASET) # LOAD_AGGREGATED_DATASET needs to be False and True if NOT high RAM available


In [6]:
if LOAD_COMPLETE_DATASET :
  print(f'reading complete dataset at: {dataset_path}')
  adata = sc.read_h5ad(dataset_path)
  vars = list(adata.var_names)
  non_gene_vars = [var for var in vars if 'ENSG' not in var]
  print(f'non_gene_vars={non_gene_vars}')
  #ordering of filtering matters
  #we should filter by cells and then genes
  sc.pp.filter_cells(adata, min_genes=150,inplace=True) # keep cells with gene exp in atleast 150 genes
  sc.pp.filter_genes(adata, min_cells=15,inplace=True)  # keep genes expressed in ≥15 cells
  remaining_genes = list(adata.var_names)
  filtered_genes = set(vars).difference(set(remaining_genes))
  print('filtered_genes:',len(filtered_genes))
  print('remaining_genes:',len(remaining_genes))

  #total count normalization
  sc.pp.normalize_total(adata,target_sum=1e6)
  # log scale data to ensure smoother distribution
  sc.pp.log1p(adata)

  #plot highly expressive genes
  sc.pl.highest_expr_genes(adata, n_top=50)

  #dimensionality reduction
  sc.pp.highly_variable_genes(adata, n_top_genes=3000, flavor='seurat',inplace=True)
  adata  = adata[:, adata.var['highly_variable']]

  #use pca
  sc.pp.pca(adata, svd_solver="arpack", use_highly_variable=True)
  sc.pl.pca_scatter(adata, color="disease")
  sc.pl.pca_scatter(adata, color="tissue")
  sc.pl.pca_scatter(adata, color="cell_type")

  #save the smaller scanpy file with highly variable genes only (hvg)
  save_subset = True
  adata  = adata[:, adata.var['highly_variable']]
  if save_subset:
    print(f'saving dataset to dataset_subpath={dataset_subpath}')
    adata.write_h5ad(dataset_subpath)
  #specify columns for our dataset
  metadata_columns = ['donor_id']
  ohe_columns = ['sex','cell_type','tissue'] #keeping for aggregation purposes
  label_column = ['disease']
  le_columns = label_column
  categorical_columns = ohe_columns
  numerical_columns = [col for col in list(adata.var_names) if 'ENSG' in col]
  #create dataframe to facilitate modeling and analysis
  gene_expr_df = pd.DataFrame(adata.X.toarray(), index=adata.obs['donor_id'], columns= numerical_columns)
  for col in categorical_columns + label_column:
    gene_expr_df[col] = adata.obs[col].values
  print(f'saving dataset to dataset_preaggregated_path={dataset_preaggregated_path}')
  gene_expr_df.to_csv(dataset_preaggregated_path,index=True)
  print(f'saving dataset from dataset_preaggregated_path={dataset_preaggregated_path}')
  #there is a bug in scanpy so its important to load file as a csv directly from pandas
  gene_expr_df = pd.read_csv(dataset_preaggregated_path)
  num_samples = gene_expr_df.shape[0]

  drop_cols = compute_drop_columns(data=gene_expr_df,columns = numerical_columns + categorical_columns,null_threshold = 0.33,n=num_samples)
  if len(drop_cols):
    print(f"drop_cols:{drop_cols}")
    gene_expr_df.drop(drop_cols,axis=1,inplace=True)
  #aggregating dataset to reduce sparsity issue and allow for compute efficiency
  group_cols = ['donor_id','disease']
  print(f'categorical_columns:{categorical_columns} numerical_columns:{numerical_columns}')
  agg_expr_df = gene_expr_df.groupby(group_cols).mean(numerical_columns).reset_index()
  before = agg_expr_df.shape[0]
  nan_columns_agg = agg_expr_df.isna().sum()[agg_expr_df.isna().sum() > 0].index
  print(f'nan_columns:{nan_columns_agg}')
  print(f'saving aggregated dataset to {dataset_aggregated_path}')
  agg_expr_df.to_csv(dataset_aggregated_path,index=False)
  agg_expr_df
elif LOAD_PREAGGREGATED_DATASET:
  print(f'reading dataset from dataset_preaggregated_path={dataset_preaggregated_path}')
  gene_expr_df = pd.read_csv(dataset_preaggregated_path)
  metadata_columns = ['donor_id']
  ohe_columns = ['sex','cell_type','tissue'] #keeping for aggregation purposes
  label_column = ['disease']
  le_columns = label_column
  categorical_columns = ohe_columns + le_columns
  numerical_columns = [col for col in list(gene_expr_df) if 'ENSG' in col]
  num_samples = gene_expr_df.shape[0]
  drop_cols = compute_drop_columns(data=gene_expr_df,columns = numerical_columns + categorical_columns,null_threshold = 0.33,n=num_samples)
  if len(drop_cols):
    print(f"drop_cols:{drop_cols}")
    gene_expr_df.drop(drop_cols,axis=1,inplace=True)

  #aggregating dataset to reduce sparsity issue and allow for compute efficiency
  group_cols = ['donor_id','disease']
  print(f'categorical_columns:{categorical_columns} numerical_columns:{numerical_columns}')
  agg_expr_df = gene_expr_df.groupby(group_cols).mean(numerical_columns).reset_index()
  before = agg_expr_df.shape[0]
  nan_columns_agg = agg_expr_df.isna().sum()[agg_expr_df.isna().sum() > 0].index
  print(f'nan_columns:{nan_columns_agg}')
  print(f'saving aggregated dataset to {dataset_aggregated_path}')
  assert all([True if 'ENSG' in col else False for col in numerical_columns ])
  agg_expr_df.to_csv(dataset_aggregated_path,index=False)
  agg_expr_df
elif LOAD_AGGREGATED_DATASET:
  print(f'reading aggregated dataset from {dataset_aggregated_path}')
  #load aggregated dataset
  agg_expr_df = pd.read_csv(dataset_aggregated_path)
  #set dataset specific respective columns
  metadata_columns = ['donor_id']
  ohe_columns = ['sex','cell_type','tissue'] #keeping for aggregation purposes
  label_column = ['disease']
  le_columns = label_column
  categorical_columns = ohe_columns + le_columns
  #ensg are gene ids
  numerical_columns = [col for col in list(agg_expr_df) if 'ENSG' in col]
  assert all([True if 'ENSG' in col else False for col in numerical_columns ])




### Split Dataset

In [7]:
import xgboost as xgb
from sklearn.model_selection import StratifiedGroupKFold,GroupKFold,GroupShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

### Stratified split

In [8]:
if not LOAD_TRAIN_TEST_DATASET:
  print('running train/test splits...')
  ohe_columns = ['sex','cell_type','tissue'] #keeping for aggregation purposes
  if dataset_type == 'cancer':
    ignore_cols = ['cell_type','sex','tissue','development_stage']
    ohe_columns = []
    print(f'ignoring columns={ignore_cols} for {dataset_type} dataset ')
    agg_expr_df.drop(ignore_cols,axis=1,inplace=True)
    ohe_columns = list(set(ohe_columns).difference(set(ignore_cols)))
  le_columns = ['disease']
  if len(ohe_columns):
    agg_expr_df,ohe_columns = process_categorical_columns(data = agg_expr_df,columns=ohe_columns,category='ohe' )
  agg_expr_df,le_columns_new = process_categorical_columns(data = agg_expr_df,columns=le_columns,category='le' )
  categorical_columns = ohe_columns
  feature_cols = list(set(agg_expr_df.columns).intersection(set(categorical_columns + numerical_columns)))
  categorical_columns = list(set(feature_cols).difference(set(numerical_columns)))
  label_columns = ['disease','disease_encoded']
  label_column = 'disease_encoded'
  assert len(set(feature_cols).intersection(set(metadata_columns))) == 0
  assert len(feature_cols) == len(categorical_columns + numerical_columns)
  print(f'categorical_cols:{len(categorical_columns)}')
  print(f'numerical_cols:{len(numerical_columns)}')
  print(f'feature_cols:{len(feature_cols)}')
  print(f'le_cols:{len(le_columns)}')
  print(f'le_cols_new:{len(le_columns_new)}')

  agg_expr_df[feature_cols + label_columns]
  # Use StratifiedGroupKFold for donor-aware stratified splitting
  splitter = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
  label_column = 'disease_encoded'
  metadata_column = 'donor_id'
  group_col = metadata_column
  y = agg_expr_df[label_column]
  groups = agg_expr_df[metadata_column]

  # Grab first split only
  for train_idx, test_idx in splitter.split(agg_expr_df, agg_expr_df[label_column], groups):
      df_train = agg_expr_df.iloc[train_idx].copy()
      df_test = agg_expr_df.iloc[test_idx].copy()
      break

  # Sanity checks
  assert not set(df_train[group_col]) & set(df_test[group_col]), "Donor overlap detected!"

  # Check label distribution
  train_dist = df_train[label_column].value_counts(normalize=True).rename("train %")
  test_dist = df_test[label_column].value_counts(normalize=True).rename("test %")
  print("Disease label proportions (train vs test):")
  print(pd.concat([train_dist, test_dist], axis=1))
  print("Train shape:", df_train.shape)
  print("Test shape:", df_test.shape)


  # -------------------------
  # STANDARDIZATION (Z-scoring only gene expression features)
  # -------------------------

  # Compute mean/std from training data
  mu = df_train[numerical_columns].mean()
  sd = df_train[numerical_columns].std().replace(0, 1)

  # Apply standardization
  df_train[numerical_columns] = (df_train[numerical_columns] - mu) / sd
  df_test[numerical_columns] = (df_test[numerical_columns] - mu) / sd


  # Save standardized splits
  print(f'saving train to {dataset_train_path}')
  print(f'saving test to {dataset_test_path}')
  df_train.to_csv(dataset_train_path, index=False)
  df_test.to_csv(dataset_test_path, index=False)
  print("Standardized gene expression features saved.")
else:
  # Load standardized splits
  print(f'Loading train from {dataset_train_path}')
  print(f'Loading test from {dataset_test_path}')



  df_train = pd.read_csv(dataset_train_path)
  df_test = pd.read_csv(dataset_test_path)


  train_groups = df_train.donor_id.unique()
  test_groups = df_test.donor_id.unique()

  assert len(set(df_train.columns).intersection(set(df_test.columns))) == len(df_train.columns)
  #need to ohe train+test combined
  agg_expr_df = pd.concat([df_train,df_test],axis=0)
  if dataset_type == 'cancer':
    ignore_cols = ['cell_type','sex','tissue','development_stage']
    ohe_columns = []
    print(f'ignoring columns={ignore_cols} for {dataset_type} dataset ')
    agg_expr_df.drop(ignore_cols,axis=1,inplace=True)
  else:
    ohe_columns = ['sex','cell_type','tissue']

  le_columns = ['disease']
  if len(ohe_columns):
    agg_expr_df,ohe_columns = process_categorical_columns(data = agg_expr_df,columns=ohe_columns,category='ohe' )
  agg_expr_df,le_columns_new = process_categorical_columns(data = agg_expr_df,columns=le_columns,category='le' )
  categorical_columns = ohe_columns
  numerical_columns = [col for col in df_train.columns if 'ENSG' in col]
  label_column = ['disease_encoded']
  label_columns = ['disease','disease_encoded']
  metadata_columns = ['donor_id']
  feature_cols = list(set(agg_expr_df.columns).intersection(set(categorical_columns + numerical_columns)))
  categorical_columns = list(set(feature_cols).difference(set(numerical_columns)))
  label_columns = ['disease','disease_encoded']
  label_column = 'disease_encoded'
  assert len(set(feature_cols).intersection(set(metadata_columns))) == 0
  assert len(feature_cols) == len(categorical_columns + numerical_columns)
  df_train , df_test = agg_expr_df[agg_expr_df.donor_id.isin(train_groups)],agg_expr_df[agg_expr_df.donor_id.isin(test_groups)]



train_groups = df_train['donor_id']
X_train,X_test = df_train[feature_cols],df_test[feature_cols]
y_train,y_test = df_train[label_columns],df_test[label_columns]


print('train:',X_train.shape,y_train.shape)
print('test:',X_test.shape,y_test.shape)



Loading train from /content/data/train_cancer.csv
Loading test from /content/data/test_cancer.csv
ignoring columns=['cell_type', 'sex', 'tissue', 'development_stage'] for cancer dataset 
train: (839, 1500) (839, 2)
test: (227, 1500) (227, 2)


In [9]:
num_classes = len(y_train[label_column].unique())
num_features = len(feature_cols)
print(f'num_featiures={num_features}')
print(f'num_classes={num_classes}')
assert len(feature_cols) == X_train.shape[1]

num_featiures=1500
num_classes=8


In [10]:
if len(list(set(X_train.columns).intersection(set(metadata_columns)))):
  print(f'dropping columns={metadata_columns} from train')
  X_train.drop(metadata_columns,axis=1,inplace=True)
  print(X_train.shape,y_train.shape)

if len(list(set(X_test.columns).intersection(set(metadata_columns)))):
  print(f'dropping columns={metadata_columns} from test')
  X_test.drop(metadata_columns,axis=1,inplace=True)
  print(X_test.shape,y_test.shape)